In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('./total_data.csv')
monthly = pd.read_csv('./month_stock.csv')
stocks = pd.read_csv('./trade_train_set/stocks.csv')
trade_train = pd.read_csv('./trade_train_set/trade_train.csv')

In [3]:
len(data[data['20년7월TOP3대상여부']=='Y']['종목번호'].unique())

133

In [4]:
stocks = stocks.drop(['index'], axis=1)
trade_train = trade_train.drop(['Unnamed: 0'], axis=1)

In [5]:
'''****** 변수 수정

거래량, 가격 -> 1을 기준으로 scaling, or 전월을 기준으로 +a%
20년도 7월에 포함되지 않은 것은 모두 drop


****** 확실히 버릴 것
그룹내_매수여부
그룹내_매도여부
기준년월
평균매수수량
평균매도수량
매수가격_중앙값
매도가격_중앙값
20년 7월 top3
종목명


******* 애매한 것
매도고객수 (전월 데이터를 사용할까?) -> 사용한다면 19년 7월은 어떻게 할까?

산업 대분류

산업 중분류

산업 소분류



******* 가져갈 것.

그룹번호
그룹내 고객수
종목번호
매수고객수 (Y)
거래량
거래금액_만원단위(정규화)
시장구분(정규화)
평균가격(정규화)
직전달 TOP3 여부'''

'****** 변수 수정\n\n거래량, 가격 -> 1을 기준으로 scaling, or 전월을 기준으로 +a%\n20년도 7월에 포함되지 않은 것은 모두 drop\n\n\n****** 확실히 버릴 것\n그룹내_매수여부\n그룹내_매도여부\n기준년월\n평균매수수량\n평균매도수량\n매수가격_중앙값\n매도가격_중앙값\n20년 7월 top3\n종목명\n\n\n******* 애매한 것\n매도고객수 (전월 데이터를 사용할까?) -> 사용한다면 19년 7월은 어떻게 할까?\n\n산업 대분류\n\n산업 중분류\n\n산업 소분류\n\n\n\n******* 가져갈 것.\n\n그룹번호\n그룹내 고객수\n종목번호\n매수고객수 (Y)\n거래량\n거래금액_만원단위(정규화)\n시장구분(정규화)\n평균가격(정규화)\n직전달 TOP3 여부'

In [6]:
data = data[data['20년7월TOP3대상여부'] == 'Y']

data = data.drop(['그룹내_매수여부', '그룹내_매도여부', '평균매수수량', '평균매도수량', '매수가격_중앙값', '매도가격_중앙값', 
          '20년7월TOP3대상여부', '종목명'], axis=1)

In [7]:
dat_mean = data[['거래금액_만원단위', '평균가격', '거래량']].mean(axis=0)

dat_std = data[['거래금액_만원단위', '평균가격', '거래량']].std()

from sklearn.preprocessing import StandardScaler
data[['거래금액_만원단위', '평균가격', '거래량']] = StandardScaler().fit_transform(data[['거래금액_만원단위', '평균가격', '거래량']])

data

,기준년월,그룹번호,그룹내고객수,종목번호,매수고객수,매도고객수,거래량,거래금액_만원단위,시장구분,표준산업구분코드_대분류,표준산업구분코드_소분류,표준산업구분코드_중분류,평균가격
0,201907,MAD01,288,A000660,7,17,-0.205595,0.229490,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,-0.111103
1,201907,MAD04,776,A000660,23,36,-0.205595,0.229490,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,-0.111103
2,201907,MAD05,890,A000660,3,11,-0.205595,0.229490,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,-0.111103
3,201907,MAD06,1615,A000660,27,63,-0.205595,0.229490,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,-0.111103
4,201907,MAD09,634,A000660,7,13,-0.205595,0.229490,코스피,제조업,반도체 제조업,전자부품· 컴퓨터· 영상· 음향 및,-0.111103
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30162,202006,MAD43,2879,A009240,6,3,-0.588937,-0.599464,코스피,도매 및 소매업,생활용품 도매업,도매 및 상품 중개업,0.013648
30164,202006,MAD43,2879,A011150,0,3,-0.385884,-0.611417,코스피,제조업,기타 식품 제조업,식료품 제조업,-0.750732
30167,202006,MAD43,2879,A036930,3,5,-0.530777,-0.624016,코스닥,제조업,특수 목적용 기계 제조업,기타 기계 및 장비 제조업,-0.734887
30180,202006,MAD43,2879,A158430,6,5,-0.579683,-0.614776,코스닥,정보통신업,소프트웨어 개발 및 공급업,출판업,-0.506165


In [8]:
a = trade_train[['기준년월', '그룹번호', '종목번호', '매수고객수']].groupby(['기준년월', '그룹번호','종목번호']).sum(
).sort_values(by = ['기준년월','그룹번호','매수고객수'], ascending=[True, True, False]).reset_index(drop=False)

a = a.groupby(['기준년월', '그룹번호']).head(3)

prev_month_list = list(a[a['기준년월'] == 202006]['종목번호'])

data['직전달TOP3여부'] = 0
for i in range(data.shape[0]):
    if data.iloc[i,3] in prev_month_list:
        data.iloc[i,-1] = 1

In [9]:
sum(data['직전달TOP3여부'] == 1)

4805

In [10]:
len(data[data['직전달TOP3여부'] == 1]['종목번호'].unique())

25

In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
def dummy(data,col):
    lab=LabelEncoder()
    aa=lab.fit_transform(data[col]).reshape(-1,1)
    ohe=OneHotEncoder(sparse=False)
    column_names=[col+'_'+ str(i) for i in lab.classes_]
    return(pd.DataFrame(ohe.fit_transform(aa),columns=column_names))

In [12]:
categ_columns = ['그룹번호', '종목번호', '표준산업구분코드_중분류', '표준산업구분코드_소분류', '시장구분']
n_columns = ['기준년월','그룹내고객수', '매수고객수', '거래량', '거래금액_만원단위', '평균가격', '직전달TOP3여부']

df = data[n_columns]
df.index = range(df.shape[0])

In [13]:
lab = LabelEncoder()
for col in categ_columns:
    temp_df = pd.DataFrame(lab.fit_transform(data[col]))
    temp_df.columns = [col]
    df = pd.concat([df, temp_df], axis=1)

In [14]:
df.to_csv('./df.csv',mode='w', header=True, index=False)

In [15]:
df

,기준년월,그룹내고객수,매수고객수,거래량,거래금액_만원단위,평균가격,직전달TOP3여부,그룹번호,종목번호,표준산업구분코드_중분류,표준산업구분코드_소분류,시장구분
0,201907,288,7,-0.205595,0.229490,-0.111103,1,0,3,26,19,1
1,201907,776,23,-0.205595,0.229490,-0.111103,1,3,3,26,19,1
2,201907,890,3,-0.205595,0.229490,-0.111103,1,4,3,26,19,1
3,201907,1615,27,-0.205595,0.229490,-0.111103,1,5,3,26,19,1
4,201907,634,7,-0.205595,0.229490,-0.111103,1,8,3,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7485,202006,2879,6,-0.588937,-0.599464,0.013648,0,42,26,10,22,1
7486,202006,2879,0,-0.385884,-0.611417,-0.750732,0,42,32,14,12,1
7487,202006,2879,3,-0.530777,-0.624016,-0.734887,0,42,58,7,50,0
7488,202006,2879,6,-0.579683,-0.614776,-0.506165,0,42,100,29,26,0


In [16]:
# 예측 템플릿 만들기
그룹번호 = pd.Series(df['그룹번호'].unique())
종목번호 = pd.Series(df['종목번호'].unique())


temp = []
for i in 그룹번호:
    for j in 종목번호:
        temp.append([i, j])

                                
temp = pd.DataFrame(data=temp, columns = ['그룹번호', '종목번호'])

In [17]:
a = trade_train[['그룹번호','그룹내고객수']].groupby('그룹번호').first().reset_index(drop=False)
a['그룹번호'] = lab.fit_transform(a['그룹번호'])

temp = pd.merge(temp, a, on='그룹번호')

temp = pd.merge(temp, df[['표준산업구분코드_중분류', '표준산업구분코드_소분류', '종목번호', '직전달TOP3여부', '시장구분']].drop_duplicates(), on='종목번호')

In [18]:
a = stocks[stocks['기준일자'] > 20200700].groupby(['종목번호']).mean().reset_index(drop=False)[['종목번호', '거래금액_만원단위']]
a['종목번호'] = lab.fit_transform(a['종목번호'])
b = stocks[stocks['기준일자'] > 20200700].groupby(['종목번호']).sum().reset_index(drop=False)[['종목번호','거래량']]
b['종목번호'] = lab.fit_transform(b['종목번호'])

a = pd.merge(a, b, on='종목번호')
a['평균가격'] = a['거래금액_만원단위'] / a['거래량']


In [19]:
temp = pd.merge(temp, a, on='종목번호')

temp['거래금액_만원단위'] = (temp['거래금액_만원단위'] - dat_mean[0]) / dat_std[0]
temp['평균가격'] = (temp['평균가격'] - dat_mean[1]) / dat_std[1]

temp

,그룹번호,종목번호,그룹내고객수,표준산업구분코드_중분류,표준산업구분코드_소분류,직전달TOP3여부,시장구분,거래금액_만원단위,거래량,평균가격
0,0,3,288,26,19,1,1,-0.605237,995851,-0.785513
1,3,3,776,26,19,1,1,-0.605237,995851,-0.785513
2,4,3,890,26,19,1,1,-0.605237,995851,-0.785513
3,5,3,1615,26,19,1,1,-0.605237,995851,-0.785513
4,8,3,634,26,19,1,1,-0.605237,995851,-0.785513
...,...,...,...,...,...,...,...,...,...,...
6379,28,132,115,14,17,0,0,-0.628993,19112414,-0.785520
6380,44,132,204,14,17,0,0,-0.628993,19112414,-0.785520
6381,9,132,106,14,17,0,0,-0.628993,19112414,-0.785520
6382,39,132,82,14,17,0,0,-0.628993,19112414,-0.785520


In [20]:
temp = temp[['그룹내고객수','거래량','거래금액_만원단위','평균가격','직전달TOP3여부', '그룹번호', 
      '종목번호', '표준산업구분코드_중분류', '표준산업구분코드_소분류', '시장구분']]

temp.to_csv('./template.csv',mode='w', header=True, index=False)

In [21]:
df.columns = ['date', 'custom_in_grp', 'buyers', 'volume', 'trade_total', 'avg_price', 'top3', 
             'grp', 'stock', 'mid', 'small', 'market']

df

,date,custom_in_grp,buyers,volume,trade_total,avg_price,top3,grp,stock,mid,small,market
0,201907,288,7,-0.205595,0.229490,-0.111103,1,0,3,26,19,1
1,201907,776,23,-0.205595,0.229490,-0.111103,1,3,3,26,19,1
2,201907,890,3,-0.205595,0.229490,-0.111103,1,4,3,26,19,1
3,201907,1615,27,-0.205595,0.229490,-0.111103,1,5,3,26,19,1
4,201907,634,7,-0.205595,0.229490,-0.111103,1,8,3,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7485,202006,2879,6,-0.588937,-0.599464,0.013648,0,42,26,10,22,1
7486,202006,2879,0,-0.385884,-0.611417,-0.750732,0,42,32,14,12,1
7487,202006,2879,3,-0.530777,-0.624016,-0.734887,0,42,58,7,50,0
7488,202006,2879,6,-0.579683,-0.614776,-0.506165,0,42,100,29,26,0


In [22]:
import lightgbm as lgb

train = df[df['date'] != 202006]
val = df[df['date'] == 202006]

/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightgbm/__init__.py:42: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  warnings.warn("Starting from version 2.2.1, the library file in distribution wheels for macOS "


In [23]:
x_train = train.drop(['date','buyers'], axis=1)
y_train = train['buyers']
x_val = val.drop(['date','buyers'], axis=1)
y_val = val['buyers']

x_train

,custom_in_grp,volume,trade_total,avg_price,top3,grp,stock,mid,small,market
0,288,-0.205595,0.229490,-0.111103,1,0,3,26,19,1
1,776,-0.205595,0.229490,-0.111103,1,3,3,26,19,1
2,890,-0.205595,0.229490,-0.111103,1,4,3,26,19,1
3,1615,-0.205595,0.229490,-0.111103,1,5,3,26,19,1
4,634,-0.205595,0.229490,-0.111103,1,8,3,26,19,1
...,...,...,...,...,...,...,...,...,...,...
6249,444,0.009122,-0.614219,-0.774632,0,41,71,1,51,0
6250,2879,-0.550899,-0.626664,-0.731056,0,42,35,10,14,0
6251,2879,-0.572986,-0.620988,-0.632480,0,42,46,25,8,1
6252,2879,-0.543249,-0.632570,-0.768635,0,42,76,10,14,0


In [24]:
categ=['grp', 'stock', 'mid', 'small']
train_ds = lgb.Dataset(x_train, label=y_train, categorical_feature=categ, free_raw_data=False)
val_ds = lgb.Dataset(x_val, label=y_val, categorical_feature=categ, free_raw_data=False)

In [52]:
2**20

1048576

In [56]:
def lgbm_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2284, num_rounds=3000):

    param = {}

    param['boosting'] = 'gbdt'
    
    param['objective'] = 'regression'

    param['learning_rate'] = 0.05

    param['max_depth'] = 20

    param['metric'] = 'rmse'
    
    param['min_data_in_leaf'] = 100
    
    param['is_training_metric'] = True
    


    param['bagging_fraction'] = 0.8
    
    param['num_leaves'] = 300

    param['feature_fraction'] = 0.8


    
    param['seed'] = seed_val
    
    num_rounds = num_rounds

    plst = list(param.items())

    model = lgb.train(param, train_ds, num_rounds,val_ds, verbose_eval= 100, early_stopping_rounds=180)

    return model

model = lgbm_regressor(train_X = x_train, train_y = y_train, test_X = x_val, test_y = y_val)

Training until validation scores don't improve for 180 rounds
[100]	valid_0's rmse: 30.0321
[200]	valid_0's rmse: 26.5211
[300]	valid_0's rmse: 24.7206
[400]	valid_0's rmse: 23.5483
[500]	valid_0's rmse: 22.7229
[600]	valid_0's rmse: 22.0955
[700]	valid_0's rmse: 21.6033
[800]	valid_0's rmse: 21.1407
[900]	valid_0's rmse: 20.792
[1000]	valid_0's rmse: 20.4811
[1100]	valid_0's rmse: 20.2118
[1200]	valid_0's rmse: 20.0245
[1300]	valid_0's rmse: 19.7866
[1400]	valid_0's rmse: 19.6054
[1500]	valid_0's rmse: 19.4751
[1600]	valid_0's rmse: 19.3301
[1700]	valid_0's rmse: 19.1929
[1800]	valid_0's rmse: 19.0622
[1900]	valid_0's rmse: 18.9381
[2000]	valid_0's rmse: 18.8193
[2100]	valid_0's rmse: 18.7013
[2200]	valid_0's rmse: 18.5756
[2300]	valid_0's rmse: 18.4772
[2400]	valid_0's rmse: 18.4063
[2500]	valid_0's rmse: 18.34
[2600]	valid_0's rmse: 18.2648
[2700]	valid_0's rmse: 18.203
[2800]	valid_0's rmse: 18.1337
[2900]	valid_0's rmse: 18.0791
[3000]	valid_0's rmse: 18.0379
Did not meet early st

In [26]:
temp.columns = x_train.columns

pred = model.predict(temp)

pred

array([ 6.4026116 , 22.21745969,  7.1260116 , ...,  1.477008  ,
        8.22959654,  3.34166032])

In [27]:
a = pd.DataFrame(pred, columns=['buyers'])

result = pd.concat([temp, a], axis = 1)

result

,custom_in_grp,volume,trade_total,avg_price,top3,grp,stock,mid,small,market,buyers
0,288,995851,-0.605237,-0.785513,1,0,3,26,19,1,6.402612
1,776,995851,-0.605237,-0.785513,1,3,3,26,19,1,22.217460
2,890,995851,-0.605237,-0.785513,1,4,3,26,19,1,7.126012
3,1615,995851,-0.605237,-0.785513,1,5,3,26,19,1,52.083382
4,634,995851,-0.605237,-0.785513,1,8,3,26,19,1,10.122504
...,...,...,...,...,...,...,...,...,...,...,...
6379,115,19112414,-0.628993,-0.785520,0,28,132,14,17,0,4.163913
6380,204,19112414,-0.628993,-0.785520,0,44,132,14,17,0,-0.926460
6381,106,19112414,-0.628993,-0.785520,0,9,132,14,17,0,1.477008
6382,82,19112414,-0.628993,-0.785520,0,39,132,14,17,0,8.229597


In [28]:
tmp = result[['grp', 'stock', 'buyers']].sort_values(by = ['grp','buyers'], ascending=[True, False])

tmp  = tmp.groupby(['grp']).head(3)
tmp

,grp,stock,buyers
5808,0,114,46.468537
144,0,17,11.717894
192,0,18,10.085167
5841,1,114,16.909920
177,1,17,6.018244
...,...,...,...
216,46,18,19.356207
168,46,17,19.255222
5833,47,114,19.569597
745,47,47,11.245257


In [29]:
rrr = pd.DataFrame(columns=['buyers', 'Top1','Top2', 'Top3'])

top_list = []
top = []
count = 0
for idx, col in tmp.iterrows():
    if count % 3 == 0:
        count +=1
        top.append(col['grp'].astype(int))
        top.append(col['stock'].astype(int))

    elif count % 3 == 1:
        count +=1
        top.append(col['grp'].astype(int))
    else:
        top.append(col['stock'].astype(int))
        top_list.append(top)
        #clear
        top = []
        count = 0

    

for top in top_list:
    print(top)
rrr

[0, 114, 0, 18]
[1, 114, 1, 47]
[2, 114, 2, 65]
[3, 114, 3, 18]
[4, 114, 4, 59]
[5, 114, 5, 55]
[6, 114, 6, 47]
[7, 114, 7, 40]
[8, 114, 8, 59]
[9, 114, 9, 17]
[10, 114, 10, 13]
[11, 114, 11, 19]
[12, 114, 12, 18]
[13, 114, 13, 17]
[14, 114, 14, 31]
[15, 114, 15, 59]
[16, 114, 16, 31]
[17, 114, 17, 18]
[18, 114, 18, 18]
[19, 114, 19, 13]
[20, 114, 20, 55]
[21, 114, 21, 17]
[22, 114, 22, 55]
[23, 114, 23, 47]
[24, 114, 24, 59]
[25, 114, 25, 17]
[26, 114, 26, 13]
[27, 114, 27, 17]
[28, 47, 28, 125]
[29, 114, 29, 17]
[30, 114, 30, 17]
[31, 114, 31, 17]
[32, 114, 32, 55]
[33, 114, 33, 17]
[34, 114, 34, 11]
[35, 114, 35, 17]
[36, 114, 36, 59]
[37, 114, 37, 18]
[38, 37, 38, 47]
[39, 114, 39, 59]
[40, 114, 40, 59]
[41, 114, 41, 59]
[42, 114, 42, 105]
[43, 114, 43, 18]
[44, 114, 44, 13]
[45, 114, 45, 18]
[46, 114, 46, 17]
[47, 114, 47, 13]


,buyers,Top1,Top2,Top3


In [30]:
lab1 = LabelEncoder()
lab1.fit(data['그룹번호'])
lab2 = LabelEncoder()
lab2.fit(data['종목번호'])

LabelEncoder()

In [31]:
print(lab2.inverse_transform([114 ,0, 18]))
print(lab1.inverse_transform([0]))

['A228760' 'A000100' 'A005935']
['MAD01']
